In [37]:
import os

class Country():
    """Country class which specifies the (top 10) ranking of a country.
    Optionally gives the score this country achieved in the semifinal."""
    def __init__(self, country, ranking_list, semifinal_score=0):
        self.country = country
        self.ranking_list = ranking_list
        self.semifinal_score = semifinal_score

    def get_ranking_list(self):
        return self.ranking_list

    def get_semifinal_score(self):
        return self.semifinal_score

    def get_ith_ranking(self, i):
        return self.ranking_list[i-1]

class Rule():
    """Rule class consisting of a scoring vector and a vector length.
    Scoring vector is specified from largest to smallest score.
    Will be expanded once the thesis gets to more advanced
    material."""
    def __init__(self, scoring_vector, vector_length):
        self.scoring_vector = scoring_vector
        self.vector_length = vector_length

    def get_scoring_vector(self):
        return self.scoring_vector

    def get_vector_length(self):
        return self.vector_length

class Contest():
    """Contest class includes everything to compute the original outcome
    of a certain year's ESC. Later will also be able to solve the problem(s)
    specified in the thesis proposal."""
    def __init__(self, country_list, voting_rule, nr_of_semifinals=0):
        self.country_list = country_list
        self.voting_rule = voting_rule
        self.nr_of_countries = len(country_list)
        self.nr_of_semifinals = nr_of_semifinals
        
    def compute_result(self, rule=None):
        """Computes the result and outputs a dict with each country that scored
        more than 0 points."""
        result = dict()
        if rule is None:
            rule_vector = self.voting_rule.get_scoring_vector()
            rule_length = self.voting_rule.get_vector_length()
        else:
            rule_vector = rule.get_scoring_vector()
            rule_length = rule.get_vector_length()
        for country in self.country_list:
            ranking_list = country.get_ranking_list()
            rank = 0
            for c in ranking_list:
                if rank + 1 > rule_length:
                    break
                if c not in result:
                    result[c] = 0
                result[c] += rule_vector[rank]
                rank += 1

        return result

    def print_result(self, rule=None):
        """Prints the result in a ranking list format."""
        result = self.compute_result(rule)
        for element in sorted(result, key=result.get, reverse=True):
            print(element, result[element])
        print()


In [38]:
import csv

def import_scores(range1, range2, file_name = 'votes.csv'):
    """ Reads and puts the votes for the finals
    from range1 to range2 in a dictionary. This dict has 2 levels:
    Top level: key = year, value = dict of scoring countries
    Bottom level: key = scoring country, value = ranked list of preferences (top 10)"""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC = dict()
    for year in range(range1, range2):
        ESC[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(range1, range2) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC[row[0]]:
            ESC[row[0]][row[2]] = dict()
        ESC[row[0]][row[2]][row[3]] = int(row[6])
        
    scores_to_rankings(ESC)
    return ESC

def scores_to_rankings(ESC_dict):
    """ ESC dict contains a year dict, which contains country dicts with a dict
    using scored_country/score tuples. This function turns that into a list
    ranking the countries instead. """
    for year in ESC_dict:
        for country in ESC_dict[year]:
            score_dict = ESC_dict[year][country]
            ranking_list = []
            for scored_country in sorted(score_dict, key=score_dict.get, reverse=True):
                ranking_list.append(scored_country)
            ESC_dict[year][country] = ranking_list
    pass

def dict_to_classes(d, voting_rule):
    """Takes as input a dict outputted by import_scores() and a scoring rule,
    and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(1975,2016):
        ESC_classified[str(year)] = None
        
    for year in d:
        country_list = []
        for country in d[year]:
            new_country = Country(country, d[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)
    
    return ESC_classified

In [39]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)

ESC_classified = dict_to_classes(import_scores(1975, 2016), rule_1975_2015)
ESC_classified['1975'].print_result()

nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3



In [4]:
rule_1962 = Rule([3,2,1], 3)
rule_1963 = Rule([5,4,3,2,1], 5)
rule_1964_66 = Rule([5,3,1], 3)

ESC_old = dict_to_classes(import_scores(1962, 1967), rule_1962)
ESC_old['1963'].print_result()
ESC_old['1964'].print_result()
ESC_old['1965'].print_result()
ESC_old['1966'].print_result()

### ESC whatever ###
ch 20
dk 20
it 16
gb 11
mc 9
fr 9
at 5
lu 3
be 2
de 1

### ESC whatever ###
it 30
gb 12
fr 10
mc 10
lu 9
at 7
fi 6
no 4
dk 3
nl 2
be 2
es 1

### ESC whatever ###
lu 21
gb 16
fr 15
it 11
at 10
ie 7
dk 6
ch 5
mc 5
se 4
nl 3
yu 2
no 1
pt 1

### ESC whatever ###
at 20
no 10
se 10
ie 9
be 9
ch 8
yu 6
es 6
lu 5
de 5
fi 5
gb 5
pt 4
dk 3
nl 2
fr 1



In [10]:
def import_scores_2015_and_beyond(file_name = 'votes.csv'):
    """ Hardcoded function. Reads and puts the votes for the finals
    from 2016 to 2019 in three dictionaries. Dicts have 2 levels:
    Top level: key = year, value = dict of scoring countries
    Bottom level: key = scoring country, value = ranked list of preferences (top 10)
    The dicts for the jury votes and televotes contain a ranking,
    the dict for the total score contains scores."""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC_jury, ESC_tele, ESC_total = dict(), dict(), dict()

    for year in range(2016,2020):
        ESC_jury[str(year)], ESC_tele[str(year)] = dict(), dict()
        ESC_total[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(2016,2020) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC_jury[row[0]] and row[8] != 0:
            ESC_jury[row[0]][row[2]] = dict()
        if row[2] not in ESC_tele[row[0]] and row[7] != 0:
            ESC_tele[row[0]][row[2]] = dict()
        if row[2] not in ESC_total[row[0]]:
            ESC_total[row[0]][row[2]] = dict()
        if row[8] != 0:
            ESC_jury[row[0]][row[2]][row[3]] = int(row[8])
        if row[7] != 0:
            ESC_tele[row[0]][row[2]][row[3]] = int(row[7])
        ESC_total[row[0]][row[2]][row[3]] = int(row[6])

    scores_to_rankings(ESC_jury)
    scores_to_rankings(ESC_tele)

    return ESC_jury, ESC_tele, ESC_total

def pre_2015_ify(jury, tele, total, voting_rule=Rule([12,10,8,7,6,5,4,3,2,1], 10)):
    """Converts the post 2015 total score (which has ties) to a 1975-2012 ranking.
    To break ties, the jury vote is checked: the country which is ranked higher
    in the jury vote wins the tie."""
    result = dict()
    rule_vector = voting_rule.get_scoring_vector()
    rule_length = voting_rule.get_vector_length()
    
    def swap_positions(l, elem1, elem2):
        pos1, pos2 = l.index(elem1), l.index(elem2)
        l[pos1], l[pos2] = l[pos2], l[pos1] 
    
    def bubble_sort_tuple_list(l):
        for i in range(len(l) - 1):
            if l[i][1] == l[i + 1][1]:
                if tele.index(l[i+1][0]) < tele.index(l[i][0]):
                    swap_positions(tuple_list, tuple_list[i+1], tuple_list[i])
    
    pre_2015_ranking = []
    tuple_list = []
    
    for element in sorted(total, key=total.get, reverse=True):
        tuple_list.append((element, total[element]))
        
    for i in range(5):
        bubble_sort_tuple_list(tuple_list)
    
    ranking_list = []
    for i in range(len(tuple_list)):
        ranking_list.append(tuple_list[i][0])
        
    return ranking_list
        
def dicts_to_2015_classes(d_jury, d_tele, d_total, voting_rule):
    """Takes as input two dicts outputted by import_scores_2015_and_beyond() and
    a scoring rule, and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(2016,2020):
        ESC_classified[str(year)] = None

    for year in d_jury:
        country_list = []
        for country in d_jury[year]:
            new_country = Country2015(country,
                                      d_jury[year][country], 
                                      d_tele[year][country],
                                      d_total[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)

    return ESC_classified

class Country2015(Country):
    """ranking_list contains a ranking list as specified by pre_2015_ify().
    The original scores can be computed with the scoring rule and jury and tele rankings."""
    def __init__(self, country, jury, tele, total, semifinal_score=0):
        self.country = country
        self.ranking_list_jury = jury
        self.ranking_list_tele = tele
        self.ranking_list = pre_2015_ify(jury, tele, total)
        self.semifinal_score = semifinal_score

In [11]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)
d_jury, d_tele, d_total = import_scores_2015_and_beyond()
ESC_classified_2015 = dicts_to_2015_classes(d_jury, d_tele, d_total, rule_1975_2015)

{'ua': 6, 'au': 24, 'ru': 14, 'bg': 12, 'se': 3, 'fr': 10, 'am': 2, 'pl': 5, 'lt': 4, 'mt': 2, 'il': 3, 'it': 18, 'hu': 2, 'es': 6, 'gb': 5}
['au', 'it', 'bg', 'ru', 'ua', 'pl', 'lt', 'se', 'am', 'hu', 'fr', 'mt', 'il', 'es', 'gb']
['au', 'it', 'ru', 'bg', 'fr', 'ua', 'es', 'pl', 'gb', 'lt', 'se', 'il', 'am', 'hu', 'mt']
{'ua': 10, 'au': 5, 'ru': 14, 'bg': 7, 'se': 2, 'fr': 19, 'pl': 1, 'be': 4, 'mt': 13, 'at': 4, 'hu': 3, 'ge': 18, 'cy': 12, 'es': 3, 'cz': 1}
['ru', 'ua', 'ge', 'fr', 'cy', 'mt', 'at', 'hu', 'se', 'pl', 'au', 'bg', 'be', 'es', 'cz']
['fr', 'ge', 'ru', 'mt', 'cy', 'ua', 'bg', 'au', 'at', 'be', 'hu', 'es', 'se', 'pl', 'cz']
{'ua': 10, 'ru': 5, 'bg': 18, 'fr': 13, 'lt': 8, 'be': 24, 'nl': 3, 'mt': 6, 'at': 3, 'il': 10, 'es': 7, 'hr': 1, 'gb': 8}
['be', 'bg', 'ua', 'fr', 'mt', 'ru', 'gb', 'at', 'es', 'lt', 'nl', 'il', 'hr']
['be', 'bg', 'fr', 'ua', 'il', 'gb', 'lt', 'es', 'mt', 'ru', 'at', 'nl', 'hr']
{'ua': 10, 'au': 15, 'ru': 11, 'bg': 5, 'se': 15, 'fr': 8, 'am': 2, 'pl'

In [7]:
print(ESC_classified_2015)

{'2016': <__main__.Contest object at 0x7f203634f208>, '2017': <__main__.Contest object at 0x7f203634fb70>, '2018': <__main__.Contest object at 0x7f203636b5f8>, '2019': <__main__.Contest object at 0x7f203636bf28>}


In [42]:
def compute_scores_for_rules(ESC_dict, rules):
    """Takes as input a dict of ESC's and a list of rules.
    Outputs a list with a dict of ESC's for each rule.
    Optionally prints the scores for each rule."""
    
    for rule in rules:
        for contest in ESC_dict:
            print("### " + str(contest) + " normal ###")
            ESC_dict[contest].print_result()
            print("### " + str(contest) + " new rule ###")
            ESC_dict[contest].print_result(rule)
        
    

In [43]:
rules_list = [Rule([3,2,1], 3), Rule([5,4,3,2,1], 5), Rule([5,3,1], 3)]
compute_scores_for_rules(ESC_classified, rules_list)

### 1975 normal ###
nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3

### 1975 new rule ###
nl 27
gb 22
it 15
fr 11
fi 10
lu 8
ch 6
ie 5
pt 3
il 2
se 2
es 1
mt 1
de 1

### 1976 normal ###
gb 164
fr 147
mc 93
ch 91
at 80
il 77
it 69
be 68
nl 56
ie 54
fi 44
pt 24
gr 20
lu 17
de 12
es 11
yu 10
no 7

### 1976 new rule ###
gb 33
fr 25
it 10
at 7
ch 6
mc 6
be 6
ie 6
il 4
pt 3
gr 1
nl 1

### 1977 normal ###
fr 136
gb 121
ie 119
mc 96
gr 92
ch 71
be 69
de 55
es 52
fi 50
il 49
nl 35
it 33
no 18
pt 18
lu 17
at 11
se 2

### 1977 new rule ###
gb 22
fr 20
ie 20
mc 10
gr 9
ch 7
be 6
fi 4
nl 3
de 3
il 2
lu 1
it 1

### 1978 normal ###
il 157
be 125
fr 119
mc 107
ie 86
de 84
lu 73
gr 66
ch 65
es 65
gb 61
it 53
nl 37
se 26
at 14
dk 13
pt 5
fi 2
tr 2

### 1978 new rule ###
il 27
be 17
fr 13
ie 12
mc 11
lu 9
de 8
gr 5
ch 4
es 4
it 4
nl 3
se 2
gb 1

### 1979 normal ###
il 125
es 116
fr 106
de 86
ie 80
dk 76
gb 73
gr 69
pt 64
ch 60
no 57
nl 

mt 2
gb 1
si 1
de 1
is 1

### 2015 normal ###
se 365
ru 303
it 292
be 217
au 196
lv 186
ee 106
no 102
il 97
rs 53
ge 51
az 49
me 44
si 39
ro 35
am 34
al 34
lt 30
gr 23
hu 19
es 15
cy 11
pl 10
gb 5
fr 4

### 2015 new rule ###
se 65
ru 44
it 39
au 17
be 15
lv 12
az 7
me 6
rs 5
al 5
ge 4
no 4
gr 3
ee 3
am 3
ro 3
cy 2
hu 1
il 1
si 1

### 1975 normal ###
nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3

### 1975 new rule ###
nl 54
gb 46
it 34
fr 31
fi 21
lu 20
ch 18
se 16
ie 14
es 6
il 6
mt 5
pt 5
de 3
be 2
no 2
yu 2

### 1976 normal ###
gb 164
fr 147
mc 93
ch 91
at 80
il 77
it 69
be 68
nl 56
ie 54
fi 44
pt 24
gr 20
lu 17
de 12
es 11
yu 10
no 7

### 1976 new rule ###
gb 63
fr 50
ch 22
mc 22
it 22
il 18
be 18
at 18
ie 13
fi 7
pt 6
nl 6
gr 3
lu 2

### 1977 normal ###
fr 136
gb 121
ie 119
mc 96
gr 92
ch 71
be 69
de 55
es 52
fi 50
il 49
nl 35
it 33
no 18
pt 18
lu 17
at 11
se 2

### 1977 new rule ###
gb 44
fr 44
ie 40
mc 25
gr 22

is 47
am 41
gb 23
ee 19
de 18
lt 17
fr 14
fi 13
es 8
ie 5

### 2013 new rule ###
dk 91
az 80
ua 73
no 46
ru 45
gr 41
it 40
nl 30
mt 26
hu 20
ge 13
ro 13
md 12
am 10
be 10
se 9
by 7
is 6
ee 5
fr 3
gb 2
es 1
de 1
lt 1

### 2014 normal ###
at 290
nl 238
se 218
am 174
hu 143
ua 113
ru 89
no 88
es 74
dk 74
ro 72
fi 72
ch 64
pl 62
is 58
by 43
gb 40
de 39
me 37
gr 35
it 33
az 33
mt 32
sm 14
si 9
fr 2

### 2014 new rule ###
at 107
nl 87
se 70
am 49
hu 35
ru 26
ua 26
ro 18
no 14
me 13
dk 13
by 11
pl 11
az 11
es 10
it 10
is 9
fi 7
ch 7
de 6
gb 5
mt 4
gr 3
si 3

### 2015 normal ###
se 365
ru 303
it 292
be 217
au 196
lv 186
ee 106
no 102
il 97
rs 53
ge 51
az 49
me 44
si 39
ro 35
am 34
al 34
lt 30
gr 23
hu 19
es 15
cy 11
pl 10
gb 5
fr 4

### 2015 new rule ###
se 136
ru 102
it 95
be 52
au 48
lv 37
ee 16
az 15
me 13
no 13
al 11
rs 10
ge 9
il 9
gr 7
am 6
lt 5
ro 5
cy 4
si 4
hu 3

### 1975 normal ###
nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 1

fr 3
al 3
dk 2
fi 1

### 2010 normal ###
de 246
tr 170
ro 162
dk 149
az 145
be 143
am 141
gr 140
ge 136
ua 108
ru 90
fr 82
rs 72
il 71
es 68
al 62
ba 51
pt 43
is 41
no 35
md 27
cy 27
ie 25
by 18
gb 10

### 2010 new rule ###
de 64
tr 34
dk 31
gr 26
az 26
be 23
ru 21
ro 21
am 20
ge 15
rs 13
ba 9
al 9
ua 8
il 8
es 6
by 5
cy 5
md 3
fr 2
is 1
pt 1

### 2011 normal ###
az 221
it 189
se 185
ua 159
dk 134
ba 125
gr 120
ie 119
ge 110
de 107
gb 100
md 97
si 96
rs 85
fr 82
ru 77
ro 77
at 64
lt 63
is 61
fi 57
hu 53
es 50
ee 44
ch 19

### 2011 new rule ###
az 40
it 36
se 31
ba 30
ua 26
dk 25
ie 24
ge 23
gr 20
si 16
ro 14
fr 13
lt 13
is 10
es 10
rs 8
gb 8
fi 8
md 8
de 7
hu 6
at 5
ru 3
ch 3

### 2012 normal ###
se 372
ru 259
rs 214
az 150
al 146
ee 120
tr 112
de 110
it 101
es 97
md 81
mk 71
ro 71
lt 70
cy 65
ua 65
gr 64
ba 55
ie 46
is 46
mt 41
fr 21
dk 21
hu 19
gb 12
no 7

### 2012 new rule ###
se 108
rs 46
az 34
al 31
ru 28
de 16
ee 16
mk 14
tr 12
cy 12
gr 11
es 10
ro 8
md 7
lt 6
ba 6
ua 5
it 3
ie 3